In [1]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import torch
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
from models.utils import Normalization
import fastwer
import contextlib


ModuleNotFoundError: No module named 'pytorch_lightning'

In [1]:
class CNN_biLSTM(nn.Module):
    def __init__(self,
                 spec_layer,
                 norm_mode,
                 input_dim,
                 hidden_dim=1024,
                 output_dim=88):
        super().__init__()
        
        self.spec_layer = spec_layer
        self.norm_layer = Normalization(mode=norm_mode)
        
        self.cnn = nn.Sequential(
            # layer 0
            nn.Conv2d(1, hidden_dim // 4, (3, 3), padding=1),
            nn.BatchNorm2d(hidden_dim // 4),
            nn.ReLU(),
            # layer 1
            nn.Conv2d(hidden_dim // 4, hidden_dim // 4, (3, 3), padding=1),
            nn.BatchNorm2d(hidden_dim // 4),
            nn.ReLU(),
            # layer 2
            nn.MaxPool2d((1, 2)),
            nn.Dropout(0.25),
            nn.Conv2d(hidden_dim // 4, hidden_dim // 2, (3, 3), padding=1),
            nn.BatchNorm2d(hidden_dim // 2),
            nn.ReLU(),
            # layer 3
            nn.MaxPool2d((1, 2)),
            nn.Dropout(0.25),
        )
        self.fc = nn.Sequential(
            nn.Linear((hidden_dim // 2) * (input_dim // 4), hidden_dim),
            nn.Dropout(0.5)
        )
        
        self.bilstm = nn.LSTM(hidden_dim, hidden_dim//2, batch_first=True, num_layers=1, bidirectional=True)
        
        self.classifier = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        spec = self.spec_layer(x) # (B, F, T)
        spec = torch.log(spec+1e-8)
        spec = spec.transpose(1,2) # (B, T, F)
        spec = self.norm_layer(spec)
        spec = spec.unsqueeze(1) # (B, 1, T, F)

        x = self.cnn(spec) # (B, hidden_dim//8, T, F//4)
        x = x.transpose(1,2).flatten(2)
        x = self.fc(x) # (B, T, hidden_dim//8*F//4)
        x, _ = self.bilstm(x)
        
        pred = self.classifier(x)
        
        
        return pred       
        
class CNN_LSTM(nn.Module):
    def __init__(self,
                 spec_layer,
                 norm_mode,
                 input_dim,
                 hidden_dim=1024,
                 output_dim=88):
        super().__init__()
        
        self.spec_layer = spec_layer
        self.norm_layer = Normalization(mode=norm_mode)
        
        self.cnn = nn.Sequential(
            # layer 0
            nn.Conv2d(1, hidden_dim // 4, (3, 3), padding=1),
            nn.BatchNorm2d(hidden_dim // 4),
            nn.ReLU(),
            # layer 1
            nn.Conv2d(hidden_dim // 4, hidden_dim // 4, (3, 3), padding=1),
            nn.BatchNorm2d(hidden_dim // 4),
            nn.ReLU(),
            # layer 2
            nn.MaxPool2d((1, 2)),
            nn.Dropout(0.25),
            nn.Conv2d(hidden_dim // 4, hidden_dim // 2, (3, 3), padding=1),
            nn.BatchNorm2d(hidden_dim // 2),
            nn.ReLU(),
            # layer 3
            nn.MaxPool2d((1, 2)),
            nn.Dropout(0.25),
        )
        self.fc = nn.Sequential(
            nn.Linear((hidden_dim // 2) * (input_dim // 4), hidden_dim),
            nn.Dropout(0.5)
        )
        
        self.bilstm = nn.LSTM(hidden_dim, hidden_dim//2, batch_first=True, num_layers=1, bidirectional=False)
        
        self.classifier = nn.Linear(hidden_dim //2 , output_dim)
        
    def forward(self, x):
        spec = self.spec_layer(x) # (B, F, T)
        spec = torch.log(spec+1e-8)
        spec = spec.transpose(1,2) # (B, T, F)
        spec = self.norm_layer(spec)
        spec = spec.unsqueeze(1) # (B, 1, T, F)

        x = self.cnn(spec) # (B, hidden_dim//8, T, F//4)
        x = x.transpose(1,2).flatten(2)
        x = self.fc(x) # (B, T, hidden_dim//8*F//4)
        x, _ = self.bilstm(x)
        
        pred = self.classifier(x)
        
        return pred       
        
        


Note: you may need to restart the kernel to use updated packages.


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: pytorch-lightning


In [ ]:
#TRAIN THE MODEL
model = CNN_biLSTM(spec_layer, norm_mode, input_dim = 512, hidden_dim = 1024, output_dim =88 )
model.train()

In [ ]:
from nnAudio import Spectrogram
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from utils.text_processing import TextTransform, data_processing

# For loading the output class ddictionary
import pickle
# Loading dataset
#TODO: train_dataset = 
#TODO: test_dataset =
train_dataset, valid_dataset = random_split(train_dataset, [190, 20], generator=torch.Generator().manual_seed(0))

# Create dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=128,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=128,
                             shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_dataset, batch_size=128,
                              shuffle=True, collate_fn=collate_batch)
   
#TODO: SpecLayer = getattr(Spectrogram)

#TODO: trainer = pl.Trainer() create a trainer, should have argument inside
trainer.fit(model, train_loader, valid_loader)

In [ ]:
# Define training parameters
learning_rate = 0.001
epochs = 1000
torch.manual_seed(28)
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def check_accuracy(model, data):
    
    # Compute accuracy
    
            
    return null

In [ ]:
#training
losses = []
accuracies = []

for epoch in range(epochs):
    total_loss = 0
    
    for context, target in data:
        
        # Prepare data :Spectrogram 
        
        
        # Forward pass
        model.zero_grad()
        output = model(ids)
        # Reshape to cover for absence of minibatches (needed for loss function)
        
        loss = loss_func(output, target)
        
        # Backward pass and optim
        loss.backward()
        optimizer.step()
        
        # Loss update
        total_loss += loss.data.item()
    
    # Display
    if epoch % 10 == 0:
        accuracy = check_accuracy(model, data)
        print("Accuracy after epoch {} is {}".format(epoch, accuracy))
        accuracies.append(accuracy)
        losses.append(total_loss)